<div class="well well-lg"><h1>Comparison of modified methods to calculate sea level contributions on regional grid</h1></div>

This notebook calulates sea-level contributions from an extended ISMIP6 projections, 
- using different methods (defined in slc)
- for the southpolar ISMIP region

contact: 
- Torsten Albrecht (torsten.albrecht@pik-potsdam.de) | Potsdam Institute for Climate Impact Research (PIK) and 
- Heio Goelzer (HEIG@norceresearch.no) | Norwegian Research Centre (NORCE), Affiliated Bjerknes Centre for Climate Research

<div class="alert alert-info">
CC-BY: Creative Commons Attribution 4.0 International  
</div>

In [ ]:
import matplotlib.pylab as plt
from matplotlib import cm, colors
import netCDF4 as nc
import numpy as np
import os, glob

# SL methods
from slc import slc_vaf
from slc import slc_G2020_publ
from slc import slc_G2020
from slc import slc_A2020

# Get constants
from slc import sl_constants as c

# Plotting
from slc import plotting

In [ ]:
# Main data path
#datapath = ".."
datapath = "/data/projects/ismip7sealevel/files"

In [ ]:
# Relative reference frame for Vaf and G2020
def get_slc_estimates(H0,H,B0,B,S0,S,zn0,zn,A):

    slc_af = slc_vaf.get_slc_vaf_HBA(H0,H,B0,B,A)
    slc_a2020 = slc_A2020.get_slc_A2020(H0,H,B0,B,S0,S,A)
    slc_g2020 = slc_G2020_publ.get_slc_G2020(H0,H,B0,B,zn0,zn,A)
    slc_diff_ag  = slc_a2020-slc_g2020
    slc_diff_va  = slc_af-slc_a2020
    slc_diff_vg  = slc_af-slc_g2020

    print('sea level change due to VAF:\t\t\t',np.around(slc_af,decimals=4))
    print('sea level change due to A2020:\t\t\t',np.around(slc_a2020,decimals=4))
    print('sea level change due to G2020:\t\t\t',np.around(slc_g2020,decimals=4))
    print('difference vaf-a2020:\t\t\t',np.around(slc_diff_va,decimals=4))
    print('difference vaf-g2020:\t\t\t',np.around(slc_diff_vg,decimals=4))
    print('difference a2020-g2020:\t\t\t',np.around(slc_diff_ag,decimals=4))
    
    return (slc_af,slc_g2020,slc_a2020,slc_diff_ag,slc_diff_vg)


# Absolute reference frame for Vaf and G2020
def get_slc_estimates_abs(H0,H,B0,B,S0,S,A):

    slc_af = slc_vaf.get_slc_vaf(H0,H,B0,B,S0,S,A)
    slc_a2020 = slc_A2020.get_slc_A2020(H0,H,B0,B,S0,S,A)
    slc_g2020 = slc_G2020.get_slc_G2020(H0,H,B0,B,A)
    slc_diff_ag  = slc_a2020-slc_g2020
    slc_diff_vg  = slc_af-slc_g2020
    slc_diff_va  = slc_af-slc_a2020

    print('sea level change due to VAF:\t\t\t',np.around(slc_af,decimals=4))
    print('sea level change due to A2020:\t\t\t',np.around(slc_a2020,decimals=4))
    print('sea level change due to G2020:\t\t\t',np.around(slc_g2020,decimals=4))
    print('difference vaf-a2020:\t\t\t',np.around(slc_diff_va,decimals=4))
    print('difference vaf-g2020:\t\t\t',np.around(slc_diff_vg,decimals=4))
    print('difference a2020-g2020:\t\t\t',np.around(slc_diff_ag,decimals=4))
    
    return (slc_af,slc_g2020,slc_a2020,slc_diff_ag,slc_diff_vg,slc_diff_va)


In [ ]:
# ISMIP6 2300 contribution by PIK_PISM

fref =datapath+"/ismip2300-coupled/pism_vilma_8km_1850.nc"
#
f0 =datapath+"/ismip2300-coupled/pism_vilma_8km_1850.nc"
#f0 =datapath+"/ismip2300-coupled/pism_vilma_8km_2000.nc"
#f0 =datapath+"/ismip2300-coupled/pism_vilma_8km_2100.nc"
#f0 =datapath+"/ismip2300-coupled/pism_vilma_8km_2200.nc"
#
#f =datapath+"/ismip2300-coupled/pism_vilma_8km_2200.nc"
f =datapath+"/ismip2300-coupled/pism_vilma_8km_2300.nc"

idat = nc.Dataset(fref, 'r')
topg_ref_PISM  = idat.variables["topg"][0,:]
rsl_ref_PISM  = idat.variables["rslc"][0,:]
x_ref_PISM  = idat.variables["x"][:]
y_ref_PISM  = idat.variables["y"][:]
idat.close()
vx, vy = np.meshgrid(x_ref_PISM, y_ref_PISM, indexing='xy') #2d\n

idat = nc.Dataset(f0, 'r')
lithk0_PISM  = idat.variables["lithk"][0,:]
topg0_PISM  = idat.variables["topg"][0,:]
rsl0_PISM  = idat.variables["rslc"][0,:]
ur0_PISM  = idat.variables["ur"][0,:]
idat.close()
H0 = lithk0_PISM
B0 = topg_ref_PISM + ur0_PISM
S0 = B0 - topg0_PISM
Zn0 = -S0

idat = nc.Dataset(f, 'r')
lithk_PISM  = idat.variables["lithk"][0,:]
topg_PISM  = idat.variables["topg"][0,:]
rsl_PISM  = idat.variables["rslc"][0,:]
ur_PISM  = idat.variables["ur"][0,:]
idat.close()
# in PISM topg=topg_ref-rsl; to determine changes in B and S separately use ur
# B=topg_ref_PISM+ur_PISM
# S=
H = lithk_PISM
B = topg_ref_PISM + ur_PISM
S = B - topg_PISM
Zn = -S

# compare changes to initial and reference state in lower left corner
print(topg_PISM[0,0]-topg0_PISM[0,0], rsl_PISM[0,0]-rsl0_PISM[0,0], B[0,0]-B0[0,0], S[0,0])
print(topg_PISM[0,0]-topg_ref_PISM[0,0], rsl_PISM[0,0]-rsl_ref_PISM[0,0], B[0,0]-B0[0,0], S[0,0])
print(B[0,0]-B0[0,0], S[0,0]-S0[0,0], S[0,0]-S0[0,0]-(B[0,0]-B0[0,0] ), rsl_PISM[0,0]-rsl0_PISM[0,0])


In [ ]:
# grid generic info

picafile = datapath+"/ismip2300-coupled/pism_8km_cellarea.nc"
vdat = nc.Dataset(picafile, 'r')
ca = vdat.variables["cell_area"][:]
vdat.close()

A = ca

print('ice volume 0', np.sum(H0*A),'m3')
print('ice volume 1', np.sum(H*A),'m3')


In [ ]:
# Compare reference frame

##print('# # # ')
#print('PISM-VILMA rel:')
#get_slc_estimates(H0,H,B0,B,0,0,0,0,A)

##print('# # # ')
#print('PISM-VILMA abs:')
#get_slc_estimates_abs(H0,H,B0,B,0,0,A)
#print('# Same results for absolute and relative formulation of Vaf and G2020 # ')

# Compare methods
print('# PISM grid # \n')
print('PISM surface: ', np.round(np.sum(A)*1e-12,decimals=4),'million km2\n')
print('Vtot  :\t\t\t',np.around(slc_vaf.get_slc_vtot(H0,H,A),decimals=4))
print('Vgr  :\t\t\t',np.around(slc_vaf.get_slc_vgr(H0,H,B0,B,S0,S,A),decimals=4))
print('Vaf  :\t\t\t',np.around(slc_vaf.get_slc_vaf(H0,H,B0,B,S0,S,A),decimals=4),'\n')
print('A2020:\t\t\t',np.around(slc_A2020.get_slc_A2020(H0,H,B0,B,S0,S,A),decimals=4))
print('G2020:\t\t\t',np.around(slc_G2020.get_slc_G2020(H0,H,B0,B,A),decimals=4))


For comparison, global and regional values:

### remapping bias

Sea-level contribution estimated on stereographic regional grid are about 5 cm lower than on regional lon-lat grid.

In [ ]:
print("A_reg_n256-A_reg_8km:",np.around(35.9504-36.0287,decimals=4),'million km2')
print("A2020_reg_n256-A2020_reg_8km:",np.around(2.2636-2.2123,decimals=4))
print("G2020_reg_n256-G2020_reg_8km:",np.around(2.358-2.3055,decimals=4))

In [ ]:
plotting.plot_regional(S-S0, vx, vy, 'sea level fingerprint in m')

In [ ]:
plotting.plot_regional(B-B0, vx, vy, 'vertical land motion in m')

In [ ]:
plotting.plot_regional(rsl_PISM-rsl0_PISM, vx, vy, 'relative sea level change in m')

In [ ]:
# assuming no change in sea level or bedrock

# Compare methods
print('# PISM grid # \n')
print('PISM surface: ', np.round(np.sum(A)*1e-12,decimals=4),'million km2\n')
print('Vtot  :\t\t\t',np.around(slc_vaf.get_slc_vtot(H0,H,A),decimals=4))
print('Vgr  :\t\t\t',np.around(slc_vaf.get_slc_vgr(H0,H,B0,B0,S0,S0,A),decimals=4))
print('Vaf  :\t\t\t',np.around(slc_vaf.get_slc_vaf(H0,H,B0,B0,S0,S0,A),decimals=4),'\n')
print('A2020:\t\t\t',np.around(slc_A2020.get_slc_A2020(H0,H,B0,B0,S0,S0,A),decimals=4))
print('G2020:\t\t\t',np.around(slc_G2020.get_slc_G2020(H0,H,B0,B0,A),decimals=4))


G2020 and A2020 are identical, and 7 cm larger than VAF!

In [ ]:
plotting.plot_regional(H-H0, vx, vy, 'ice thickness change in m')

### subgrid mask information

In [ ]:
# considering subgrid mask information, assuming L=G, B=B0, S=S0

I0,L0,G0 = slc_A2020.get_masks_A2020(H0,B0,S0)
G0bin=G0

#print('PISM initial binary land area: ', np.round(np.sum(L0*A)*1e-12,decimals=4),'million km2')
#print('PISM initial binary grounded ice area: ', np.round(np.sum(G0*A)*1e-12,decimals=4),'million km2')
#print('L0-G0: ', np.round((np.sum(L0*A)-np.sum(G0*A))*1e-12,decimals=4),'million km2\n')

idat = nc.Dataset(f0, 'r')
G0  = idat.variables["sftgrf"][0,:]
I0  = idat.variables["sftgif"][0,:]
idat.close()

#print('PISM initial fractional grounded ice area: ', np.round(np.sum(G0*A)*1e-12,decimals=4),'million km2')
#print('G0_sub-G0_bin: ', np.round((np.sum(G0*A)-np.sum(G0bin*A))*1e-12,decimals=4),'million km2\n')

I,L,G = slc_A2020.get_masks_A2020(H,B,S)
Gbin=G

print('PISM 2300 binary land area: ', np.round(np.sum(L*A)*1e-12,decimals=4),'million km2')
print('PISM 2300 binary grounded ice area: ', np.round(np.sum(G*A)*1e-12,decimals=4),'million km2')
print('L-G: ', np.round((np.sum(L*A)-np.sum(G*A))*1e-12,decimals=4),'million km2\n')

idat = nc.Dataset(f, 'r')
G  = idat.variables["sftgrf"][0,:]
I  = idat.variables["sftgif"][0,:]
idat.close()

print('PISM 2300 fractional grounded ice area: ', np.round(np.sum(G*A)*1e-12,decimals=4),'million km2')
print('G0_sub-G0_bin: ', np.round((np.sum(G*A)-np.sum(Gbin*A))*1e-12,decimals=4),'million km2\n')

# test initial vs. 2300 consistency
print('A2020_bin-A2020_bin\t:',np.around(slc_A2020.get_slc_A2020(H0,H,B0,B0,S0,S0,A)-slc_A2020.get_slc_A2020_with_masks(H0,H,B0,B0,S0,S0,L0,L,G0bin,Gbin,A),decimals=4))

Sub-grid information on grounded ice area fraction `G`, but binary land mask `L`. Considering fractional G mask explains less than 3 mm SLC for A2020 in this example.

In [ ]:
print('A2020_sub:\t',np.around(slc_A2020.get_slc_A2020_with_masks(H0,H,B0,B0,S0,S0,L0,L,G0,G,A),decimals=4))
print('A2020-A2020_sub\t:',np.around(slc_A2020.get_slc_A2020(H0,H,B0,B0,S0,S0,A)-slc_A2020.get_slc_A2020_with_masks(H0,H,B0,B0,S0,S0,L0,L,G0,G,A),decimals=4))

### time series

In [ ]:
slc_Vaf_array=[]
slc_G2020_array=[]
slc_A2020_array=[]

snapshots=[1850,2000,2100,2200,2300]

for year in snapshots:
    #print(str(year))
    f =datapath+"/ismip2300-coupled/pism_vilma_8km_"+str(year)+".nc"

    idat = nc.Dataset(f, 'r')
    lithk_PISM  = idat.variables["lithk"][0,:]
    topg_PISM  = idat.variables["topg"][0,:]
    rsl_PISM  = idat.variables["rslc"][0,:]
    ur_PISM  = idat.variables["ur"][0,:]
    idat.close()

    H = lithk_PISM
    B = topg_ref_PISM + ur_PISM
    S = B - topg_PISM
    Zn = -S
    
    slc_Vaf_array.append(np.around(slc_vaf.get_slc_vaf(H0,H,B0,B0,S0,S0,A),decimals=4))
    slc_G2020_array.append(np.around(slc_G2020.get_slc_G2020(H0,H,B0,B,A),decimals=4))
    slc_A2020_array.append(np.around(slc_A2020.get_slc_A2020(H0,H,B0,B0,S0,S0,A),decimals=4))
        

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.05,0.05,0.9,0.9])

ax.plot(snapshots,slc_Vaf_array,color='C0',label='VAF',marker='o')
ax.plot(snapshots,slc_G2020_array,color='C1',label='G2020',marker='x')
ax.plot(snapshots,slc_A2020_array,color='C2',label='A2020',marker='+')

ax.set_ylabel('sea-level contribution in m SLE')
plt.legend()
ax.set_xlim(1850,2300)
ax.set_ylim(0,2.5)